In [ ]:
import json
#from google.colab import drive

# Get the notebook's filename (usually matches the GitHub repo name)
!ls *.ipynb
notebook_name = "NLPProject.ipynb"  # ← Replace with your filename

# Load and fix the notebook
with open(notebook_name, 'r') as f:
    nb = json.load(f)

# Option A: Remove widgets metadata completely (recommended)
if 'metadata' in nb and 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Option B: Or add the missing state key
# if 'metadata' in nb and 'widgets' in nb['metadata']:
#     nb['metadata']['widgets']['state'] = {}

# Save the fixed version
with open(notebook_name, 'w') as f:
    json.dump(nb, f)

print("Notebook metadata fixed! You can now commit to GitHub.")

ModuleNotFoundError: No module named 'google'

In [6]:
from pprint import pprint
from operator import itemgetter
import nltk
from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList
nltk.download('framenet_v17')


[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/kierstenwener/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


True

Overview Notes

-FE is a frame element, the the frame evpking words are the lexical units (LU)
    -the concept of cooking involves the cook, the food that is being cooked, something holding the food while cooking, and the source of heat
    -this is represented by a frame called Apply_Heat and the Cook, Food, Heating_Instrument, and COntainer are the FEs; the words that evoke the frame (like fry, bake, boil, broil, etc) are the LUs

-inheritance: An IS-A relationship, the child frame is a subtype of the parent frame, each FE in the parent is bound to a corresponding FE in the child
    -ex; the "Revenge" frame which inherits from the "Rewards_and_punishments" frame
-using: The child frame presupposes the parent frame as background
    -eg: the "speed" frame "uses" the "Motion" frame, but not all parent FEs need to be bound to child FEs
-subframe: The child is a sibvent of a complex event represented by the parent
    -eg: the "Criminal_process" frame has subframes "arrest", "arraignment", "trial" and "sentencing"
-Perspective on: the child frame provices a particular perspective on an un-perspectivized parent frame; a pair of examples consists of "Hiring" and "Get_a_job" frames, which perspectivize the "Employment_start" frame from the Employers and Employees point of view

-use frames() to get list of all frames; can get particular frame and info by using frame(); can provide a regular expressions to frames() to get list of all frames whoe names match the patern
-can search Frames by their Lexical Units using frames_by_lemma(), which returns list of all frames that contain LUs in which the 'name' attrib of the LU matches the given regular expressiuon

In [111]:
f = fn.frame(202)
print(f.ID)
print(f.name)
print(f.definition)
print(f.frameRelations)

202
Arrest
Authorities charge a Suspect, who is under suspicion of having committed a crime (the Charges), and take him/her into custody.'The police arrested Harry on charges of manslaughter.'
[<Parent=Intentionally_affect -- Inheritance -> Child=Arrest>, <Complex=Criminal_process -- Subframe -> Component=Arrest>, ...]


Lexical Unit overview
- A lexical unit is a pairing of a word with a meaning
-the "Apply_Heat" frame describes a common situation involving a cook, some food, and a heating instrument and is_envoked_ by words such as bake, blanch, broil, brown, simmer, steak, etc
-The frame evoking words are the LUs in the Apply_head frame :
    -can be adjs, nouns, or verbs
    - reduction evokes "Cause_change_of_scalar_position" in .."the reduction of debt levels to 665 million from 2.6 billion"
-Framenet provides mult annotated examples of each sense of a word (i.e each LU)
-each LU is linked to a frame, and the other words that evoke that frame
    -framenet groups similar words together sematnically, like a thesaurous

In [112]:
#this is how you get details for a specific lexical unit
PrettyList(sorted(fn.lus(r'(?i)a little'), key=itemgetter('ID')))

[<lu ID=14733 name=a little.n>, <lu ID=14743 name=a little.adv>, ...]

-the LU named takes the form of the dotten string in which the lemme preceeds the"." and a part of speech follows the "."

In [113]:
print(fn.lu(256).name)
print(fn.lu(256).definition)
print(fn.lu(256).frame.name)
print(fn.lu(256).lexemes[0].name)

foresee.v
COD: be aware of beforehand; predict.
Expectation
foresee


get an annotated document with the docs() function

In [114]:
d = fn.docs('BellRinging')[0]
print(d.corpname)
print(d.sentence[49])

PropBank
full-text sentence (2824246) in BellRinging:


[POS] 17 tags

[POS_tagset] PENN

[text] + [annotationSet]

`` I live in hopes that the ringers themselves will be drawn into
             *****          *******                    *****     
             Desir          Cause_t                    Cause     
             [1]            [3]                        [2]       

 that fuller life .
      ******
      Comple
      [4]   
 (Desir=Desiring, Cause_t=Cause_to_make_noise, Cause=Cause_motion, Comple=Completeness)



Below, you get information about the first annotation in the specificed sentence above. We see that the word hope evokes the frame desiring

In [115]:
d.sentence[49].annotationSet[1]

annotation set (4528852):

[status] MANUAL

[LU] (6605) hope.n in Desiring

[frame] (366) Desiring

[GF] 2 relations

[PT] 2 phrases

[text] + [Target] + [FE] + [Noun]

`` I live in hopes that the ringers themselves will be drawn into
   - ^^^^ ^^ ***** ----------------------------------------------
   E supp su       Event                                         
 
 that fuller life .
-----------------
                 
 (E=Experiencer, su=supp)


find the training data :
-this is a whole set of the training data, can index it to look at specific frames

In [116]:
frames = fn.frames()
print(f"Number of frames: {len(frames)}")

Number of frames: 1221


-frame elements are the specific participants, roles, or components of a frame;
for each frame element,how many sample sentences do we have?

In [117]:
frame_element_counts = {}
#for each frame, loops through all frame elements
for frame in fn.frames():
    frame_name = frame.name

    for fe_name, fe in frame.FE.items():

        sample_sentences = frame.lexUnit
        num_sentences = len(sample_sentences)

        # Store the count of sentences for each frame element
        if fe_name in frame_element_counts:
            frame_element_counts[fe_name] += num_sentences  # Add the new count to the existing one
        else:
            frame_element_counts[fe_name] = num_sentences

sorted_frame_elements = sorted(frame_element_counts.items(), key=lambda x: x[1], reverse=True)
for fe_name, count in sorted_frame_elements[:10]:
    print(f"Frame Element: {fe_name}, Sample Sentences: {count}")

Frame Element: Time, Sample Sentences: 8170
Frame Element: Manner, Sample Sentences: 7612
Frame Element: Place, Sample Sentences: 7037
Frame Element: Degree, Sample Sentences: 7012
Frame Element: Means, Sample Sentences: 5045
Frame Element: Explanation, Sample Sentences: 4539
Frame Element: Depictive, Sample Sentences: 4091
Frame Element: Purpose, Sample Sentences: 4091
Frame Element: Circumstances, Sample Sentences: 3219
Frame Element: Duration, Sample Sentences: 3120


now we can say, given a sentence, which word corresponds to the "" element

-we could have a target frame element, get all of the sentences associated with that frame element.
-First, we need to associate the sentence with a semantic frame, then parse to get the frame elements, or use bert to predict which word corresponds to a specific frame element

https://arxiv.org/html/2502.12516v1
-Json based formats show superior results, wrap target or word phrase in **


In [118]:
from transformers import pipeline

In [119]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should return True if you have a CUDA-compatible GPU


2.6.0+cu124
False


### TAGGING

Started tagging using the listed top ten words. Added a way for context to be acknowledged while attempting to tag.

In [190]:


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import framenet as fn
nltk.download('averaged_perceptron_tagger_eng')
import re
from nltk import pos_tag
from transformers import BertTokenizer, BertModel

#  pre-trained BERT model and tokenizer ( this will provide context for example sentences)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

top_10_fes = [
    "Time", "Manner", "Place", "Degree", "Means", "Explanation",
    "Depictive", "Purpose", "Circumstances", "Duration"
]

# dictionary to store lexical units for the top 10 frame elements
frame_element_lexemes = {}

# function to strip POS ( parts-of-speech) tags from lexemes
def strip_pos_tag(lexeme):
    return re.sub(r'\.[a-z]+$', '', lexeme)

# loop through all frames in FrameNet to gather lexical units for each of the top 10 frame elements
for frame in fn.frames():
    frame_name = frame.name  # Frame name (e.g., "Cause", "Motion")

    # loop through each frame element in the frame
    for fe_name, fe in frame.FE.items():
        if fe_name in top_10_fes:
            # get all lexical units for this frame element
            if fe_name not in frame_element_lexemes:
                frame_element_lexemes[fe_name] = set()
            # if it's a lexical unit object, access its lexemes
            for lu in frame.lexUnit:
                if hasattr(lu, 'lexemes'):
                    for lexeme in lu.lexemes:
                        lexeme_name = strip_pos_tag(lexeme.name.lower())  # remove POS tag
                        frame_element_lexemes[fe_name].add(lexeme_name)  # store lexemes in lowercase without POS tag
                else:
                    # if it's just a string, we directly add it as a lexeme
                    lexeme_name = strip_pos_tag(lu.lower())  # remove POS tag
                    frame_element_lexemes[fe_name].add(lexeme_name)

# function to get BERT embeddings for a sentence
def get_bert_embeddings(sentence):
    # tokenize and encode sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    # get the embeddings from BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # return the token embeddings
    return outputs.last_hidden_state

# function to tag a sentence with context
def tag_sentence_with_context(sentence):
    tokens = word_tokenize(sentence)
    token_embeddings = get_bert_embeddings(sentence)

    tagged_sentence = []
    pos_tags = pos_tag(tokens)  # POS tagging to help with filtering

    # loop through each word in the sentence
    for i, (word, pos) in enumerate(pos_tags):
        word_lower = strip_pos_tag(word.lower())  # Convert word to lowercase and strip POS tag
        assigned_fe = "none"

        # check each top frame element for the word
        for fe_name, lexemes in frame_element_lexemes.items():
            if word_lower in lexemes:
                # use BERT embeddings and POS tags to determine if the word is related to the correct FE
                embedding = token_embeddings[0, i].cpu().numpy()

                # apply conditions to check if the word is related to the correct FE based on its embedding and POS tag
                if fe_name == "Time" and pos in ['NN', 'NNS', 'VB', 'VBP']:  # time is often linked to nouns/verbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Place" and pos in ['NN', 'NNS']:  # place-related words are usually nouns
                    assigned_fe = fe_name
                    break
                elif fe_name == "Degree" and pos in ['RB', 'JJ']:  # degree-related words are often adverbs or adjectives
                    assigned_fe = fe_name
                    break
                elif fe_name == "Manner" and pos in ['RB']:  # manner-related words are adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Means" and pos in ['NN', 'NNS', 'IN']:  # means-related words are nouns or prepositions
                    assigned_fe = fe_name
                    break
                elif fe_name == "Purpose" and pos in ['NN', 'VB', 'IN']:  # purpose-related words are nouns, verbs, or prepositions
                    assigned_fe = fe_name
                    break
                elif fe_name == "Explanation" and pos in ['NN', 'VB']:  # explanation-related words are nouns or verbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Circumstances" and pos in ['NN', 'IN', 'JJ', 'RB']:  # circumstances-related words are nouns, prepositions, adjectives, or adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Depictive" and pos in ['JJ', 'RB']:  # depictive-related words are adjectives or adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Duration" and pos in ['NN', 'VB', 'JJ', 'RB']:  # duration-related words are nouns, verbs, adjectives, or adverbs
                    assigned_fe = fe_name
                    break

        tagged_sentence.append((word, assigned_fe))

    return tagged_sentence

# Example sentences for testing
sentences = [
    "The meeting will begin at 3 PM.",
    "She worked all night to finish the project.",
    "The concert starts tomorrow.",
    "She explained the concept clearly.",
    "The team worked together efficiently.",
    "He ran away nervously.",
    "They visited the park in the morning.",
    "I left my keys on the kitchen counter.",
    "The store is near the bus station.",
    "It was so hot that I couldn’t stand outside.",
    "He’s extremely talented.",
    "The cake was quite delicious.",
    "She achieved her goals through hard work.",
    "We traveled by car across the country.",
    "The results were obtained using advanced techniques.",
    "He explained why he was late for the meeting.",
    "The teacher clarified the rules of the game.",
    "She gave a detailed explanation about the new policy.",
    "She looked tired after the long trip.",
    "The sky was clear and blue.",
    "The dog seemed excited and playful.",
    "He went to the store to buy groceries.",
    "The project was created to improve customer service.",
    "She studies hard to become a doctor.",
    "Despite the rain, they went for a walk.",
    "Under normal circumstances, the event would be postponed.",
    "In the event of a fire, use the emergency exit.",
    "He lived in Paris for two years.",
    "The lecture lasted for three hours.",
    "She was absent for a week due to illness."
]

# process each sentence and tag it
tagged_sentences = []

for sentence in sentences:
    tagged = tag_sentence_with_context(sentence)
    tagged_sentences.append(tagged)

# print and test sentences
for sentence, tagged in zip(sentences, tagged_sentences):
    print(f"Original Sentence: {sentence}")
    print(f"Tagged Sentence: {tagged}")
    print("-" * 50)




[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Original Sentence: The meeting will begin at 3 PM.
Tagged Sentence: [('The', 'none'), ('meeting', 'Place'), ('will', 'none'), ('begin', 'Time'), ('at', 'Purpose'), ('3', 'none'), ('PM', 'none'), ('.', 'none')]
--------------------------------------------------
Original Sentence: She worked all night to finish the project.
Tagged Sentence: [('She', 'none'), ('worked', 'none'), ('all', 'none'), ('night', 'none'), ('to', 'none'), ('finish', 'Time'), ('the', 'none'), ('project', 'Place'), ('.', 'none')]
--------------------------------------------------
Original Sentence: The concert starts tomorrow.
Tagged Sentence: [('The', 'none'), ('concert', 'none'), ('starts', 'none'), ('tomorrow', 'none'), ('.', 'none')]
--------------------------------------------------
Original Sentence: She explained the concept clearly.
Tagged Sentence: [('She', 'none'), ('explained', 'none'), ('the', 'none'), ('concept', 'none'), ('clearly', 'Degree'), ('.', 'none')]
--------------------------------------------